In [ ]:
from deep_orderbook.replayer import Replayer
from deep_orderbook.shapper import BookShapper
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
MARKETS = ["ETHBTC", "BTCUSDT", "ETHUSDT", "BNBBTC", "BNBETH", "BNBUSDT"]

In [ ]:
# itertools for asyncio !
import asyncio
import itertools
import aioitertools

In [ ]:
shapper = await BookShapper.create()
file_replayer = Replayer('../data/crypto', date_regexp='20')
areplay = file_replayer.replayL2_async(pair='BTCUSDT', shapper=shapper)

In [ ]:
file_gen = file_replayer.file_generator(pair='BTCUSDT')

i = 0
async for _ in file_gen:
    i += 1
    if i >= 10:
        break

In [ ]:
import sys
sys.version

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
g = file_replayer.training_files('BTCUSDT', side_bips=8, side_width=32)
next(g)

In [ ]:
def gen(frac_from=0.0, frac_to=1.0):
    file_names = list(file_replayer.training_files('BTCUSDT', side_bips=8, side_width=32))
    num = len(file_names)
    rangefrom = int(num * frac_from)
    rangeto = int(num * frac_to)
    for fn_bs, fn_ps, fn_ts in file_names[rangefrom:rangeto]:
        print(fn_bs, fn_ps, fn_ts)
        arr_books, arr_prices, arr_time2level = np.load(fn_bs), np.load(fn_ts), np.load(fn_ps)
        assert arr_books.shape[0] ==  arr_prices.shape[0]
        assert arr_books.shape[0] ==  arr_time2level.shape[0]
        yield arr_books, arr_prices, arr_time2level
        
g = gen(frac_to=0.75)
next(g)
g = gen(frac_from=0.75)
[a.shape for a in next(g)]

In [ ]:
dataset = tf.data.Dataset.from_generator( 
     gen, 
     (tf.float32, tf.float32, tf.float32), 
     (tf.TensorShape([None, 64, 3]), tf.TensorShape([None, 64, 1]), tf.TensorShape([None, 2, 3])))
 
list(dataset.take(3).as_numpy_iterator()) 

In [ ]:
# await next(file_gen)

In [ ]:
rep = file_replayer.replayL2_async('BTCUSDT', shapper)

for i in range(100):
    await next(rep)

In [ ]:
print(file_replayer.zipped_dates())

In [ ]:
for i in range(100):
    batptr = await next(areplay)
print(f"bids:\n{batptr['bids'].head()}")
print(f"asks:\n{batptr['asks'].head()}")
print(f"prices:\n{batptr['price']}")
print(f"trades:\n{batptr['trades']}")

In [ ]:
replayers = [file_replayer.replayL2_async(pair, await BookShapper.create()) for pair in MARKETS]
multi_replay = file_replayer.multireplayL2_async(replayers)
d = await next(multi_replay)

In [ ]:
async def run(maxpoints=100):
    async for i in iter(range(maxpoints)):
        d = await next(multi_replay)
    #return d

task = asyncio.create_task(run())

In [ ]:
await task

In [ ]:
genarr = shapper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await next(genarr)

In [ ]:
genacc = shapper.accumulate_array(genarr, markets=MARKETS)
_ = await next(genacc)

In [ ]:
every = 10
LENGTH = 128
x = []
async for n,sec in enumerate(genacc):
    allim = []
    for symb, data in sec.items():
        arr = np.stack(data['bs'][-LENGTH:])
        im = arr
        im[:,:,0] /= 10
        im += 0.5
        allim.append(im)
    allim = np.concatenate(allim, axis=1)
    if n % every == 0:
        plt.figure(figsize=(15,11))
        toshow = allim.transpose(1,0,2)
        toshow = np.clip(toshow, 0, 1)
        plt.imshow(toshow, origin="lower");
        plt.show()
#        pr = np.stack(data['ps'])[-LENGTH:, :-1]
#        plt.plot(pr)
#        plt.show()
#        print(symb)
        clear_output(wait=True)
#        plt.show()
#    if n == 100:
#        break
    pass